In [196]:
 import pandas as pd
 import numpy as np

# Convert to CSV

In [197]:
xlspath = '/Users/rderi/Programming/Class/Predictive Analytics/final project/FReD-2024-11-29.xlsx'
xls = pd.read_excel(xlspath)

In [198]:
csvpath = '/Users/rderi/Programming/Class/Predictive Analytics/final project/FReD-2024-11-29.csv'

In [199]:
xls.to_csv(csvpath, index=False)

# Data cleanup

In [200]:
data_raw = pd.read_csv(csvpath)

In [201]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data_raw.isna().sum())

id                                            0
temp                                        232
status                                        0
cos                                         503
validated                                     0
validated_person                              2
source                                        0
discipline                                  137
effect                                      346
tags                                        157
description                                   1
claim_text_orig                             503
claim_statistics_orig                       503
claim_statistics_orig_page                  503
claim_text_orig_page                        503
notes                                       180
contributors                                  3
date_entered                                  0
date_updated                                505
notes_validation                            421
exclusion                               

In [202]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data_raw.describe())

           status  cos  validated  claim_statistics_orig_page  \
count  505.000000  2.0      505.0                    2.000000   
mean     2.029703  1.0        1.0                  367.500000   
std      0.169935  0.0        0.0                    0.707107   
min      2.000000  1.0        1.0                  367.000000   
25%      2.000000  1.0        1.0                  367.250000   
50%      2.000000  1.0        1.0                  367.500000   
75%      2.000000  1.0        1.0                  367.750000   
max      3.000000  1.0        1.0                  368.000000   

       claim_text_orig_page  date_entered  date_updated  exclusion  \
count              2.000000    505.000000           0.0        0.0   
mean             367.500000  45187.051161           NaN        NaN   
std                0.707107    205.965444           NaN        NaN   
min              367.000000  44927.000000           NaN        NaN   
25%              367.250000  45016.000000           NaN        N

In [203]:
for col in data_raw.columns:
    print(f'Column {col}:')
    print(f'\tNumber of unique values: {data_raw[col].nunique()}')
    if data_raw[col].nunique() < 10:
        print(f'\tUnique values: {data_raw[col].unique()}')
        print(f'\tValue counts: {data_raw[col].value_counts()}\n')
    else:
        print(f'Many unique values, not displaying.')


Column id:
	Number of unique values: 389
Many unique values, not displaying.
Column temp:
	Number of unique values: 164
Many unique values, not displaying.
Column status:
	Number of unique values: 2
	Unique values: [2. 3.]
	Value counts: status
2.0    490
3.0     15
Name: count, dtype: int64

Column cos:
	Number of unique values: 1
	Unique values: [nan  1.]
	Value counts: cos
1.0    2
Name: count, dtype: int64

Column validated:
	Number of unique values: 1
	Unique values: [1]
	Value counts: validated
1    505
Name: count, dtype: int64

Column validated_person:
	Number of unique values: 7
	Unique values: ['CD' 'LKasseckert' 'Veronica Boyce' 'LK, LR' 'LK' 'LR' nan 'LK asseckert']
	Value counts: validated_person
LK                166
Veronica Boyce    121
CD                109
LKasseckert        48
LR                 43
LK, LR             15
LK asseckert        1
Name: count, dtype: int64

Column source:
	Number of unique values: 14
Many unique values, not displaying.
Column discipline:
	

In [204]:
#let me know if this covers all we'll need
relevant_cols = [
    'discipline', 
    #'effect', 
    'tags', 
    'es_original', 
    #'es_rep', 
    'n_original', 
    #'n_replication', 
    'p_value_original', 'vi_original',
    #'p_value_replication', 
    'result',
    'ci.lower_original', 'ci.upper_original',
    #'ci.lower_replication', 'ci.upper_replication'
]

In [205]:
optional_cols = [
    
    #Few rows include these
    'significant_original',
    'power'
    #'significant_replication'
    
]

In [206]:
data = data_raw[relevant_cols]

In [207]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   discipline         368 non-null    object 
 1   tags               348 non-null    object 
 2   es_original        505 non-null    float64
 3   n_original         504 non-null    float64
 4   p_value_original   504 non-null    float64
 5   vi_original        504 non-null    float64
 6   result             505 non-null    object 
 7   ci.lower_original  504 non-null    float64
 8   ci.upper_original  504 non-null    float64
dtypes: float64(6), object(3)
memory usage: 35.6+ KB


## Fixing discipline
*one-hot encoding*

In [208]:
print(data['discipline'].value_counts())

discipline
Psychology                      183
Social Psychology                71
Marketing                        61
Social psychology                20
Cognitive Psychology              6
Developmental Psychology          5
Judgment and Decision Making      4
Experimental Philosophy           4
Organizational Psychology         3
Consumer Psychology               3
Experimental philosophy           2
Cognitive psychology              2
Meta Science                      1
Human Genetics                    1
Health Psychology                 1
Applied Linguistics               1
Name: count, dtype: int64


In [209]:
print(data['discipline'].str.lower().value_counts())

discipline
psychology                      183
social psychology                91
marketing                        61
cognitive psychology              8
experimental philosophy           6
developmental psychology          5
judgment and decision making      4
organizational psychology         3
consumer psychology               3
meta science                      1
human genetics                    1
health psychology                 1
applied linguistics               1
Name: count, dtype: int64


In [210]:
data['discipline'] = data['discipline'].str.lower()

/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/2757682168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['discipline'] = data['discipline'].str.lower()


I wonder if it's worth including some of these disciplines with <=3 occurences. It might not lead to very accurate predictions, so I'm removing those rows.

In [211]:
removed_disciplines = ['applied linguistics',
                       'health psychology',
                       'human genetics',
                       'meta science',
                       'consumer psychology',
                       'organizational psychology']
data.drop(data[data['discipline'].isin(removed_disciplines)].index, inplace=True)

/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/3479711683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(data[data['discipline'].isin(removed_disciplines)].index, inplace=True)


In [212]:
dummies = pd.get_dummies(data['discipline'], prefix='disc', dtype=np.int64)
data.drop('discipline', axis=1, inplace=True)
data = pd.concat([data, dummies], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 495 entries, 0 to 504
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   tags                               338 non-null    object 
 1   es_original                        495 non-null    float64
 2   n_original                         494 non-null    float64
 3   p_value_original                   494 non-null    float64
 4   vi_original                        494 non-null    float64
 5   result                             495 non-null    object 
 6   ci.lower_original                  494 non-null    float64
 7   ci.upper_original                  494 non-null    float64
 8   disc_cognitive psychology          495 non-null    int64  
 9   disc_developmental psychology      495 non-null    int64  
 10  disc_experimental philosophy       495 non-null    int64  
 11  disc_judgment and decision making  495 non-null    int64  
 12 

/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/581490660.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('discipline', axis=1, inplace=True)


## Fix tags
this one's going to be messier

In [213]:
all_tags = data['tags'].str.split(r'[;,]\s?', expand=True, regex=True).stack().str.lower()
all_tags.drop(all_tags[all_tags==''].index, inplace=True) #not sure why this is needed
unique_tags = all_tags.unique()
tag_value_counts = all_tags.value_counts()

In [214]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tag_value_counts)

humans                                                    94
male                                                      59
female                                                    58
adult                                                     48
attention                                                 42
sample 1                                                  24
sample 2                                                  24
social value orientation (slm)                            24
many labs 2                                               24
social value orientation (tdm)                            24
adolescent                                                21
reaction time                                             21
human behavior                                            21
nature                                                    21
science                                                   21
social science replication project                        21
visual                  

*This is one of those where there might be a lot of room for improvement*

We're dropping any tag that occurs less than 10 times. This might mean that some papers are left without tags. Since about 150 papers don't have tags anyways, it's not like it wasn't an optional feature anyways. Still, there will be some extra papers left without tags—meaning there's probably a more optimal way to find the smallest set of tags that cover all the papers (so we don't increase the number of papers without tags).

In [215]:
#Drop any tag that occurs less than 10 times
all_tags.drop(all_tags[all_tags.isin(tag_value_counts[tag_value_counts < 10].index)].index, 
              inplace=True)

In [216]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(all_tags.value_counts())

humans                                                    94
male                                                      59
female                                                    58
adult                                                     48
attention                                                 42
social value orientation (tdm)                            24
sample 2                                                  24
social value orientation (slm)                            24
many labs 2                                               24
sample 1                                                  24
nature                                                    21
adolescent                                                21
reaction time                                             21
human behavior                                            21
science                                                   21
social science replication project                        21
pattern recognition     

*Are we keeping in 'sample 1'/'sample 2' as tags? I don't see them as particularly problematic, but maybe they have less predictive use.*

*Now we actually one-hot encode the tags*

In [217]:
filtered_tags = all_tags.unique()

In [218]:
for tag in filtered_tags:
    data[f'tag_{tag}'] = data['tags'].str.contains(tag, case=False, na=False).astype(np.int64)

/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/3420353527.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[f'tag_{tag}'] = data['tags'].str.contains(tag, case=False, na=False).astype(np.int64)
/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/3420353527.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[f'tag_{tag}'] = data['tags'].str.contains(tag, case=False, na=False).astype(np.int64)
/var/folders/4d/01q6438j69781g6__rcymycw0000gn/T/ipykernel_15591/3420353527.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[f'tag_{tag}'] = data['tags'].str.contains(tag, case=False, na=False).astype(np.int64)


In [219]:
data.drop('tags', axis=1, inplace=True)

In [220]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 495 entries, 0 to 504
Data columns (total 63 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   es_original                                                 495 non-null    float64
 1   n_original                                                  494 non-null    float64
 2   p_value_original                                            494 non-null    float64
 3   vi_original                                                 494 non-null    float64
 4   result                                                      495 non-null    object 
 5   ci.lower_original                                           494 non-null    float64
 6   ci.upper_original                                           494 non-null    float64
 7   disc_cognitive psychology                                   495 non-null    int64  
 8   disc_

*Okay, so there's actually only one row left with an NA value...*

In [221]:
data.dropna(inplace=True)

In [223]:
data.columns = data.columns.str.replace(' ', '_')

In [224]:
data.to_csv('/Users/rderi/Programming/Class/Predictive Analytics/final project/FReD-2024-11-29_cleaned.csv', index=False)